1. Import Libraries

In [13]:
import pandas as pd
import csv

2. Load the EEG Data

In [14]:
import pandas as pd
import re
from collections import Counter  # (optional)

# -------- CONFIG --------
path = r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\James\James_EPOC_147367_2023.03.08T14.25.07Z.md.mc.pm.fe.bp.csv"
expected_min_cols = 170
encodings_to_try = ["utf-8-sig", "utf-8", "latin-1"]

candidates = {
    ",":       {"is_regex": False},
    ";":       {"is_regex": False},
    "\t":      {"is_regex": False},
    "|":       {"is_regex": False},
    r"\s+":    {"is_regex": True},
}

# -------- STEP 1: decode a text sample for analysis --------
raw = None
text = None
used_encoding = None

for enc in encodings_to_try:
    try:
        with open(path, "rb") as f:
            raw = f.read()
        text = raw.decode(enc, errors="strict")
        used_encoding = enc
        break
    except Exception:
        continue

if text is None:
    raise RuntimeError("Could not decode file with utf-8-sig, utf-8, or latin-1.")

lines = text.splitlines()
sample_n = min(200, len(lines))
sample_lines = lines[:sample_n]

# -------- STEP 2: detect the header line and the best delimiter --------
def field_count(line, sep, is_regex):
    if not line.strip():
        return 0
    if is_regex:
        parts = re.split(sep, line.strip())
    else:
        parts = line.strip().split(sep)
    return len(parts)

best = None  # (max_fields, line_idx, sep, is_regex)

for idx, line in enumerate(sample_lines):
    if not line.strip():
        continue
    for sep, meta in candidates.items():
        cnt = field_count(line, sep, meta["is_regex"])
        if best is None or cnt > best[0]:
            best = (cnt, idx, sep, meta["is_regex"])

if best is None:
    raise RuntimeError("Could not identify any plausible header line.")

best_count, header_idx, best_sep, best_is_regex = best

if best_count < 50:
    print(f"⚠️ Detected only {best_count} fields on line {header_idx}. Continuing anyway.")

# -------- STEP 3: read the file from the detected header line --------
read_kwargs = {
    "encoding": used_encoding,
    "header": 0,                  # the first row after skiprows is the header
    "skiprows": header_idx,       # skip everything before the detected header
    "dtype": str,                 # keep 1:1 columns
    "quotechar": '"',
    "doublequote": True,
    "na_filter": False,
    # "on_bad_lines": "skip",     # uncomment if you hit malformed rows
}

if best_is_regex:
    read_kwargs["sep"] = best_sep
    read_kwargs["engine"] = "python"
else:
    read_kwargs["sep"] = best_sep
    read_kwargs["engine"] = "c"

df = pd.read_csv(path, **read_kwargs)

# -------- STEP 4: diagnostics and hard check --------
print(f"Detected encoding: {used_encoding}")
print(f"Detected header line index: {header_idx}")
print(f"Detected separator: {'REGEX ' if best_is_regex else ''}{best_sep}")
print(f"Shape: {df.shape}")

if df.shape[1] < expected_min_cols:
    hdr_preview = sample_lines[header_idx][:200].replace("\t", "\\t")
    print(f"⚠️ Columns < {expected_min_cols}. Header preview: {hdr_preview}")

assert df.shape[1] >= expected_min_cols, (
    f"Only {df.shape[1]} columns parsed; expected ≥ {expected_min_cols} (A..FN)."
)

print(df.columns.tolist()[:12])
print(df.head(3))


Detected encoding: utf-8-sig
Detected header line index: 1
Detected separator: ,
Shape: (198935, 170)
['Timestamp', 'OriginalTimestamp', 'EEG.Counter', 'EEG.Interpolated', 'EEG.AF3', 'EEG.F7', 'EEG.F3', 'EEG.FC5', 'EEG.T7', 'EEG.P7', 'EEG.O1', 'EEG.O2']
           Timestamp  OriginalTimestamp EEG.Counter EEG.Interpolated  \
0  1678285507.659898  1678285507.661464   78.000000         0.000000   
1  1678285507.667718  1678285507.669384   79.000000         0.000000   
2  1678285507.675538  1678285507.677204   80.000000         0.000000   

       EEG.AF3       EEG.F7       EEG.F3      EEG.FC5       EEG.T7  \
0  3643.076904  4263.589844  3921.538574  4129.743652  4423.589844   
1  3629.230713  4257.436035  3923.076904  4134.871582  4423.077148   
2  3635.384521  4262.051270  3922.051270  4137.948730  4422.563965   

        EEG.P7       EEG.O1       EEG.O2       EEG.P8       EEG.T8  \
0  4365.641113  4326.666504  3973.333252  4265.641113  3736.923096   
1  4366.153809  4326.153809  3967.69

In [15]:
data = df.copy()

3. Data Preprocessing

3.1. Extract the information of the dataset

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198935 entries, 0 to 198934
Columns: 170 entries, Timestamp to POW.AF4.Gamma
dtypes: object(170)
memory usage: 258.0+ MB


In [17]:
data.columns

Index(['Timestamp', 'OriginalTimestamp', 'EEG.Counter', 'EEG.Interpolated',
       'EEG.AF3', 'EEG.F7', 'EEG.F3', 'EEG.FC5', 'EEG.T7', 'EEG.P7',
       ...
       'POW.F8.Theta', 'POW.F8.Alpha', 'POW.F8.BetaL', 'POW.F8.BetaH',
       'POW.F8.Gamma', 'POW.AF4.Theta', 'POW.AF4.Alpha', 'POW.AF4.BetaL',
       'POW.AF4.BetaH', 'POW.AF4.Gamma'],
      dtype='object', length=170)

In [18]:
pd.set_option('display.max_columns', None)  # Show every column (no truncation)
pd.set_option('display.width', 0)           # Let pandas auto-size the table to the notebook width
data.head()

,Timestamp,OriginalTimestamp,EEG.Counter,EEG.Interpolated,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.T7,EEG.P7,EEG.O1,EEG.O2,EEG.P8,EEG.T8,EEG.FC6,EEG.F4,EEG.F8,EEG.AF4,EEG.RawCq,EEG.Battery,EEG.BatteryPercent,MarkerIndex,MarkerType,MarkerValueInt,EEG.MarkerHardware,CQ.AF3,CQ.F7,CQ.F3,CQ.FC5,CQ.T7,CQ.P7,CQ.O1,CQ.O2,CQ.P8,CQ.T8,CQ.FC6,CQ.F4,CQ.F8,CQ.AF4,CQ.Overall,EQ.SampleRateQuality,EQ.OVERALL,EQ.AF3,EQ.F7,EQ.F3,EQ.FC5,EQ.T7,EQ.P7,EQ.O1,EQ.O2,EQ.P8,EQ.T8,EQ.FC6,EQ.F4,EQ.F8,EQ.AF4,MOT.CounterMems,MOT.InterpolatedMems,MOT.GyroX,MOT.GyroY,MC.Action,MC.ActionPower,MC.IsActive,PM.Engagement.IsActive,PM.Engagement.Scaled,PM.Engagement.Raw,PM.Engagement.Min,PM.Engagement.Max,PM.Excitement.IsActive,PM.Excitement.Scaled,PM.Excitement.Raw,PM.Excitement.Min,PM.Excitement.Max,PM.LongTermExcitement,PM.Stress.IsActive,PM.Stress.Scaled,PM.Stress.Raw,PM.Stress.Min,PM.Stress.Max,PM.Relaxation.IsActive,PM.Relaxation.Scaled,PM.Relaxation.Raw,PM.Relaxation.Min,PM.Relaxation.Max,PM.Interest.IsActive,PM.Interest.Scaled,PM.Interest.Raw,PM.Interest.Min,PM.Interest.Max,PM.Focus.IsActive,PM.Focus.Scaled,PM.Focus.Raw,PM.Focus.Min,PM.Focus.Max,FE.BlinkWink,FE.HorizontalEyesDirection,FE.UpperFaceAction,FE.UpperFaceActionPower,FE.LowerFaceAction,FE.LowerFaceActionPower,POW.AF3.Theta,POW.AF3.Alpha,POW.AF3.BetaL,POW.AF3.BetaH,POW.AF3.Gamma,POW.F7.Theta,POW.F7.Alpha,POW.F7.BetaL,POW.F7.BetaH,POW.F7.Gamma,POW.F3.Theta,POW.F3.Alpha,POW.F3.BetaL,POW.F3.BetaH,POW.F3.Gamma,POW.FC5.Theta,POW.FC5.Alpha,POW.FC5.BetaL,POW.FC5.BetaH,POW.FC5.Gamma,POW.T7.Theta,POW.T7.Alpha,POW.T7.BetaL,POW.T7.BetaH,POW.T7.Gamma,POW.P7.Theta,POW.P7.Alpha,POW.P7.BetaL,POW.P7.BetaH,POW.P7.Gamma,POW.O1.Theta,POW.O1.Alpha,POW.O1.BetaL,POW.O1.BetaH,POW.O1.Gamma,POW.O2.Theta,POW.O2.Alpha,POW.O2.BetaL,POW.O2.BetaH,POW.O2.Gamma,POW.P8.Theta,POW.P8.Alpha,POW.P8.BetaL,POW.P8.BetaH,POW.P8.Gamma,POW.T8.Theta,POW.T8.Alpha,POW.T8.BetaL,POW.T8.BetaH,POW.T8.Gamma,POW.FC6.Theta,POW.FC6.Alpha,POW.FC6.BetaL,POW.FC6.BetaH,POW.FC6.Gamma,POW.F4.Theta,POW.F4.Alpha,POW.F4.BetaL,POW.F4.BetaH,POW.F4.Gamma,POW.F8.Theta,POW.F8.Alpha,POW.F8.BetaL,POW.F8.BetaH,POW.F8.Gamma,POW.AF4.Theta,POW.AF4.Alpha,POW.AF4.BetaL,POW.AF4.BetaH,POW.AF4.Gamma
0,1678285507.659898,1678285507.661464,78.000000,0.000000,3643.076904,4263.589844,3921.538574,4129.743652,4423.589844,4365.641113,4326.666504,3973.333252,4265.641113,3736.923096,3988.205078,4466.666504,4241.538574,4207.179688,463.000000,4.000000,100.000000,,,,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,2.000000,4.000000,4.000000,4.000000,0.000000,,,,,,,,,,,,,,,,,78.000000,0.000000,2201.000000,1474.000000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1678285507.667718,1678285507.669384,79.000000,0.000000,3629.230713,4257.436035,3923.076904,4134.871582,4423.077148,4366.153809,4326.153809,3967.692383,4265.128418,3736.923096,3982.564209,4463.589844,4238.974121,4203.077148,463.000000,4.000000,100.000000,,,,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,2.000000,4.000000,4.000000,4.000000,0.000000,,,,,,,,,,,,,,,,,79.000000,0.000000,2201.000000,1475.000000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,1678285507.675538,1678285507.677204,80.000000,0.000000,3635.384521,4262.051270,3922.051270,4137.948730,4422.563965,4368.717773,4328.717773,3968.718018,4266.153809,3736.923096,3984.615479,4471.282227,4245.641113,4211.282227,170.000000,4.000000,100.000000,,,,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,2.000000,4.000000,4.000000,4.000000,0.000000,,,,,,,,,,,,,,,,,80.000000,0.000000,2200.000000,1475.000000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,1678285507.683358,1678285507.685124,81.000000,0.000000,3670.769287,4272.307617,3920.512939,4135.8974

3.2. Keep only the columns from E-R

In [19]:
# Keep only columns E..R (Excel letters). 
# A=0,B=1,C=2,D=3,E=4,...,R=17 (0-based). iloc stop is exclusive, so use 18.
start_idx = 4           # E
stop_idx_exclusive = 18 # R + 1

# Clamp the stop index in case the file has fewer columns
stop_idx_exclusive = min(stop_idx_exclusive, data.shape[1])

data = data.iloc[:, start_idx:stop_idx_exclusive]

# (Optional) quick check
print(data.shape)

(198935, 14)


In [20]:
data.head()

,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.T7,EEG.P7,EEG.O1,EEG.O2,EEG.P8,EEG.T8,EEG.FC6,EEG.F4,EEG.F8,EEG.AF4
0,3643.076904,4263.589844,3921.538574,4129.743652,4423.589844,4365.641113,4326.666504,3973.333252,4265.641113,3736.923096,3988.205078,4466.666504,4241.538574,4207.179688
1,3629.230713,4257.436035,3923.076904,4134.871582,4423.077148,4366.153809,4326.153809,3967.692383,4265.128418,3736.923096,3982.564209,4463.589844,4238.974121,4203.077148
2,3635.384521,4262.051270,3922.051270,4137.948730,4422.563965,4368.717773,4328.717773,3968.718018,4266.153809,3736.923096,3984.615479,4471.282227,4245.641113,4211.282227
3,3670.769287,4272.307617,3920.512939,4135.897461,4424.615234,4369.743652,4328.205078,3987.179443,4265.641113,3736.923096,3998.461426,4490.769043,4271.794922,4236.410156
4,3659.487061,4265.641113,3923.076904,4129.230957,4427.692383,4367.179688,4328.717773,3976.410156,4263.589844,3736.923096,3989.743652,4475.384766,4255.897461,4221.025879


* First row is the header row. so it will not be applied in the simulation

3.3. Extract EEG data from sample (row) number 23,040 to the row number 76,800.

In [21]:
# -------- STEP 5: extract rows 23,040..76,800 (inclusive, 1-based) --------
TARGET_N = 76_800
start_row_1based = 23_040
end_row_1based   = 76_800

# Convert to 0-based, end exclusive for iloc
start_iloc = max(0, start_row_1based - 1)
n_rows = data.shape[0]
end_iloc_exclusive = min(n_rows, end_row_1based)  # 1-based inclusive -> 0-based exclusive

# Final dataset
data_sliced = data.iloc[start_iloc:end_iloc_exclusive].copy()

# Reporting
print(f"Total rows in data: {n_rows}")
if n_rows == TARGET_N:
    print("✅ Dataset has exactly 76,800 rows.")
elif n_rows > TARGET_N:
    print(f"ℹ️ Dataset has MORE than 76,800 rows: {n_rows} (+{n_rows - TARGET_N}). "
          "Printing rows beyond 76,800 as well.")
else:
    print(f"⚠️ Dataset has FEWER than 76,800 rows: {n_rows}. Slice may be shorter than requested.")

print(f"Requested rows (1-based): {start_row_1based}..{end_row_1based}")
print(f"Sliced (0-based iloc): {start_iloc}:{end_iloc_exclusive}")
print(f"data_sliced shape: {data_sliced.shape}")
if data_sliced.empty:
    print("⚠️ Slice is empty (start index is beyond available rows).")
else:
    print("Preview of requested slice:")
    print(data_sliced.head())

# If there are rows beyond 76,800, report and preview them too
if n_rows > TARGET_N:
    extra_start = TARGET_N  # 0-based iloc start for "beyond" section
    extra_end = n_rows
    extra = data.iloc[extra_start:extra_end].copy()
    print(f"\nExtra rows beyond 76,800: {n_rows - TARGET_N}")
    print(f"Extra slice (0-based iloc): {extra_start}:{extra_end} | shape: {extra.shape}")
    

Total rows in data: 198935
ℹ️ Dataset has MORE than 76,800 rows: 198935 (+122135). Printing rows beyond 76,800 as well.
Requested rows (1-based): 23040..76800
Sliced (0-based iloc): 23039:76800
data_sliced shape: (53761, 14)
Preview of requested slice:
           EEG.AF3       EEG.F7       EEG.F3      EEG.FC5       EEG.T7  \
23039  3484.102539  4068.718018  3860.512939  4197.948730  4427.692383   
23040  3484.102539  4073.333252  3861.025635  4201.538574  4427.179688   
23041  3510.256348  4089.230713  3858.974365  4202.051270  4427.179688   
23042  3518.461426  4095.384521  3856.923096  4200.000000  4424.615234   
23043  3514.358887  4098.461426  3857.435791  4201.025879  4424.102539   

            EEG.P7       EEG.O1       EEG.O2       EEG.P8       EEG.T8  \
23039  4353.333496  4327.179688  3821.538574  4319.487305  3738.974365   
23040  4351.794922  4332.820313  3825.128174  4319.487305  3738.974365   
23041  4350.256348  4335.384766  3843.076904  4319.487305  3738.974365   
23042 

3.4. Keep last 53000 rows

In [22]:
# -------- STEP 6: keep only the last 53,000 rows from data_sliced --------
TARGET_53K = 53_000

n_curr = data_sliced.shape[0]
if n_curr >= TARGET_53K:
    # number of rows to remove from the start
    n_drop = n_curr - TARGET_53K
    data_last_53k = data_sliced.iloc[-TARGET_53K:].copy()
    print(f"Trimmed {n_drop} rows from the start to keep the last {TARGET_53K}.")
else:
    # not enough rows to trim; keep as-is
    n_drop = 0
    data_last_53k = data_sliced.copy()
    print(f"Dataset has only {n_curr} rows (< {TARGET_53K}); keeping all.")

print(f"data_last_53k shape: {data_last_53k.shape}")

# (Optional) reset the index if you want a clean 0..N-1 index
data_last_53k = data_last_53k.reset_index(drop=True)


Trimmed 761 rows from the start to keep the last 53000.
data_last_53k shape: (53000, 14)


In [23]:
data_last_53k.head()

,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.T7,EEG.P7,EEG.O1,EEG.O2,EEG.P8,EEG.T8,EEG.FC6,EEG.F4,EEG.F8,EEG.AF4
0,3568.718018,4186.666504,3856.923096,4293.846191,4425.128418,4348.717773,4321.538574,3877.435791,4319.487305,3737.435791,3890.769287,4444.102539,4015.384521,4135.384766
1,3576.923096,4188.205078,3857.435791,4293.333496,4427.692383,4348.717773,4324.102539,3877.948730,4319.487305,3737.435791,3894.871826,4445.128418,4017.948730,4138.461426
2,3587.692383,4197.948730,3858.461426,4297.948730,4425.128418,4347.692383,4323.589844,3886.666748,4319.487305,3737.435791,3904.615479,4455.384766,4034.358887,4153.846191
3,3570.256348,4189.230957,3857.948730,4291.282227,4423.589844,4346.153809,4318.974121,3875.384521,4319.487305,3737.435791,3893.846191,4442.051270,4018.461426,4139.487305
4,3553.333252,4178.461426,3857.435791,4282.051270,4425.641113,4346.666504,4320.000000,3864.615479,4319.487305,3737.435791,3883.076904,4426.153809,3997.435791,4118.974121


3.5. Create csv file from filtered data

In [25]:
import os
import csv  # you already imported this above

# Ensure you've created `data_last_53k` from the previous step
# and that `path` points to your original CSV.

output_path = os.path.join(os.path.dirname(path), "james_eeg_modified.csv")

data_last_53k.to_csv(
    output_path,
    index=False,
    encoding="utf-8-sig",   # keeps Excel happy with UTF-8 BOM
    quoting=csv.QUOTE_MINIMAL
    # line_terminator="\n", # optional: normalize newlines
)

print(f"✅ Saved: {output_path} | shape: {data_last_53k.shape}")


✅ Saved: C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\James\james_eeg_modified.csv | shape: (53000, 14)


4. Create 2D array

4.1. Convert to an array

In [26]:
import numpy as np

# If you want raw strings exactly as in the CSV:
arr = data_last_53k.to_numpy(copy=True)
assert arr.shape == (53000, 14)

4.2. Transpose the array

In [27]:
# Transpose (returns a view when possible)
arr_T = arr.T
print(arr_T.shape)   # (14, 53000)

(14, 53000)


5. Segement to (14,200) shape

In [28]:
import numpy as np

seg_len = 200
channels, total_samples = arr_T.shape
n_segments = total_samples // seg_len  # 53000 // 200 = 265

# (Optional) trim if not perfectly divisible — here it is, but this keeps it robust
usable = n_segments * seg_len
arr_T_trim = arr_T[:, :usable]

# Reshape to (channels, segments, seg_len) then put segments first → (segments, channels, seg_len)
segments = arr_T_trim.reshape(channels, n_segments, seg_len).transpose(1, 0, 2)

print("segments.shape:", segments.shape)  # (265, 14, 200)


segments.shape: (265, 14, 200)


6. Add target variable

In [29]:
import numpy as np

y = np.ones(265, dtype=np.int64)   # shape (265,)

In [30]:
y.shape

(265,)

In [31]:
y[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

7. Save the arrays

7.1. Save the segments

In [32]:
import os
import numpy as np
import csv  # if you're also writing CSVs

# Assume: `path` points to your original CSV and `segments` is (265, 14, 200)
# (Optional) y is your labels vector (265,)

out_dir = os.path.dirname(path)

# ---- Save segments as a NumPy array (.npy) ----
adam_eeg_path = os.path.join(out_dir, "james_eeg.npy")
np.save(adam_eeg_path, segments)
print(f"✅ Saved segments → {adam_eeg_path} | shape: {segments.shape}")

# ---- (Optional) Save labels next to it ----



✅ Saved segments → C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\James\james_eeg.npy | shape: (265, 14, 200)


7.2. Save the target array

In [33]:
y = np.ones(segments.shape[0], dtype=np.int64)
y_path = os.path.join(out_dir, "james_eeg_labels.npy")
np.save(y_path, y)
print(f"✅ Saved labels   → {y_path} | shape: {y.shape}")

✅ Saved labels   → C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\James\james_eeg_labels.npy | shape: (265,)
